In [ ]:
import requests, PyPDF2, io
import numpy as np
import re
import pandas as pd
from sqlalchemy import create_engine

url = "https://www.spglobal.com/platts/plattscontent/_assets/_files/en/productsservices/market-reports/oilgram-proce-report-060818.pdf"
response = requests.get(url)

In [43]:
with io.BytesIO(response.content) as open_pdf_file:
    read_pdf = PyPDF2.PdfFileReader(open_pdf_file)
    num_pages = read_pdf.getNumPages()
    page = read_pdf.getPage(22)
    text = page.extractText()
    text = text.replace('Œ',' ')
    text = text.replace('/',' ')

product_mapping = pd.read_csv('product_mapping.csv')
product_mapping['Code_stop'] = product_mapping['Code'].shift(-1)

def find_price(code, code_stop):
    start = text.find(code)
    if not code_stop:
        stop = start + 30
    else:
        stop = text.find(code_stop)
    substring = text[start:stop]
    letter_pos = re.search('[A-Za-z]',substring[7:])
    if letter_pos == None:
        sub_list = substring.split()
    else:
        sub_list = substring[:(letter_pos.start()+7)].split()
    temp_list = []
    for k, st in enumerate(sub_list):
        if sub_list[k] != '':
            temp_list.append(k)
    sub_list = [sub_list[i] for i in temp_list]
    if len(sub_list) == 3:
        sub_list = [sub_list[0]] + ['',''] + sub_list[1:]
    return sub_list

product_mapping['Code_stop'] = product_mapping['Code_stop'].fillna('')
product_mapping['prices'] = product_mapping.apply(lambda x: find_price(x['Code'],x['Code_stop']), axis=1)

temp = pd.DataFrame(product_mapping.prices.tolist(), columns=['Code','Bid','Ask','Mid','Change'])
prices = product_mapping.merge(temp,how='left',on='Code').drop(['prices','Code_stop'],axis=1)
engine = create_engine('sqlite://', echo=False)

prices.to_sql('crudes', con=engine)
engine.execute("SELECT * FROM crudes").fetchall()

[(0, 'WTI', 1, 'PCACG00', '68.48', '68.50', '68.490', '-0.470'),
 (1, 'WTI', 2, 'PCACH00', '67.34', '67.36', '67.350', '-0.310'),
 (2, 'WTI', 3, 'AAGIT00', '66.92', '66.94', '66.930', '-0.250'),
 (3, 'WTI_EFP', 1, 'AAGVT00', '-0.01', '0.01', '0.000', '0.000'),
 (4, 'WTI_EFP', 2, 'AAGVU00', '-0.01', '0.01', '0.000', '0.000'),
 (5, 'WTI_EFP', 3, 'AAGVV00', '-0.01', '0.01', '0.000', '0.000'),
 (6, 'Light_Houston_', 1, 'AAXEW00', '', '', '70.890', '-0.470'),
 (7, 'Light_Houston_', 2, 'AAYRY00', '', '', '71.050', '-0.310'),
 (8, 'LOOP_Sour', 1, 'AALSM01', '', '', '68.190', '-0.470'),
 (9, 'LOOP_Sour', 2, 'AALSM02', '', '', '68.200', '-0.060'),
 (10, 'LOOP_Sour', 3, 'AALSM03', '', '', '68.280', '0.000'),
 (11, 'Eagle_Ford_Marker', 1, 'AAYAJ00', '', '', '71.000', '-0.350'),
 (12, 'Mars', 1, 'AAMBR00', '68.48', '68.50', '68.490', '-0.470'),
 (13, 'Mars', 2, 'AAMBU00', '68.49', '68.51', '68.500', '-0.060'),
 (14, 'Mars', 3, 'AAMBX00', '68.57', '68.59', '68.580', '0.000'),
 (15, 'Mars/WTI', 1, '